In [3]:
import pandas as pd
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.feature_selection import RFE

In [4]:
dataset = pd.read_csv("CKD.csv")
dataset


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [5]:
dataset.shape

(399, 25)

In [6]:
dataset.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [7]:
print(dataset['rbc'].value_counts())
print(dataset['pc'].value_counts())
print(dataset['pcc'].value_counts())
print(dataset['ba'].value_counts())
print(dataset['htn'].value_counts())
print(dataset['dm'].value_counts())
print(dataset['cad'].value_counts())
print(dataset['appet'].value_counts())
print(dataset['pe'].value_counts())
print(dataset['ane'].value_counts())
print(dataset['classification'].value_counts())
print(dataset['sg'].value_counts())

rbc
normal      352
abnormal     47
Name: count, dtype: int64
pc
normal      323
abnormal     76
Name: count, dtype: int64
pcc
notpresent    357
present        42
Name: count, dtype: int64
ba
notpresent    377
present        22
Name: count, dtype: int64
htn
no     253
yes    146
Name: count, dtype: int64
dm
no     263
yes    136
Name: count, dtype: int64
cad
no     365
yes     34
Name: count, dtype: int64
appet
yes     316
poor     83
Name: count, dtype: int64
pe
poor    322
yes      77
Name: count, dtype: int64
ane
no     339
yes     60
Name: count, dtype: int64
classification
yes    249
no     150
Name: count, dtype: int64
sg
a    152
c     84
b     81
d     75
e      7
Name: count, dtype: int64


In [8]:
dataset = pd.get_dummies(dataset, drop_first=True)
dataset=dataset.astype(int)
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2,76,3,0,148,57,3,137,4,12,...,0,0,0,0,0,0,1,1,0,1
1,3,76,2,0,148,22,0,137,4,10,...,1,0,0,0,0,0,1,0,0,1
2,4,76,1,0,99,23,0,138,4,12,...,1,0,0,0,0,0,1,0,0,1
3,5,76,1,0,148,16,0,138,3,8,...,1,0,0,0,0,0,1,0,1,1
4,5,50,0,0,148,25,0,137,4,11,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51,70,0,0,219,36,1,139,3,12,...,1,0,0,0,0,0,1,0,0,1
395,51,70,0,2,220,68,2,137,4,8,...,1,0,0,1,1,0,1,0,1,1
396,51,70,3,0,110,115,6,134,2,9,...,1,0,0,1,1,0,0,0,0,1
397,51,90,0,0,207,80,6,142,5,8,...,1,0,0,1,1,0,1,0,1,1


In [9]:
independent = dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependent = dataset[['classification_yes']]

In [12]:
print(independent.shape)
print(dependent.shape)

(399, 27)
(399, 1)


In [13]:
#split model for training and testing
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(independent, dependent, test_size=1/3, random_state=0)
X_train

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,rbc_normal,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes
240,60,100,0,3,263,27,1,135,4,12,...,1,1,0,0,1,1,1,1,0,0
218,57,80,0,0,120,48,1,137,4,11,...,1,1,0,0,1,1,0,1,0,0
101,42,70,0,0,75,31,1,141,3,16,...,1,1,0,0,0,0,0,1,0,0
311,66,70,1,0,248,30,1,138,5,12,...,1,1,0,0,1,1,0,1,0,0
194,55,80,0,0,146,57,3,137,4,9,...,1,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,68,70,0,0,98,86,4,135,3,9,...,1,1,0,0,1,1,1,0,1,0
192,54,90,1,0,150,18,1,140,4,12,...,1,0,0,0,0,0,0,0,1,1
117,45,80,0,0,107,15,1,141,4,11,...,1,0,0,0,0,0,0,1,0,0
47,30,80,0,0,89,42,0,139,5,16,...,1,1,0,0,0,0,0,1,0,0


In [14]:
# RFE
def rfefeature(indep_X,dep_Y,n):
    rfelist =[]
    selected_features = {}

    log_model = LogisticRegression(solver='lbfgs')
    svm_model = SVC(kernel='linear',random_state =0)
    dt_model = DecisionTreeClassifier(criterion ='gini',max_features ='sqrt',splitter ='best',random_state =0)
    rf_model = RandomForestClassifier(n_estimators =10, criterion ='entropy', random_state =0)
#     nb_model = GaussianNB(priors=None)
    rfemodellist = [log_model,svm_model,dt_model,rf_model]
    for model in rfemodellist:
        print(model)
        # https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
        log_rfe = RFE(estimator=model, n_features_to_select=n)
        print(log_rfe)
        log_fit = log_rfe.fit(indep_X,dep_Y)
        log_rfe_feature= log_fit.transform(indep_X)
        selected_columns = indep_X.columns[log_rfe.support_].tolist()
        selected_features[model.__class__.__name__] = selected_columns
        rfelist.append(log_rfe_feature)
    return rfelist,selected_features

In [15]:
def split_scalar(indep_X,dep_Y):
    X_train,X_test,Y_train,Y_test = train_test_split(indep_X,dep_Y,test_size=0.30,random_state=0)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train,X_test,Y_train,Y_test

In [16]:
def cm_pred(classifier,X_test):
    test_pred = classifier.predict(X_test)
    
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(Y_test,test_pred)
    
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    
    accuracy = accuracy_score(Y_test,test_pred)
    
    report = classification_report(Y_test,test_pred)
    return classifier,accuracy,report,X_test,Y_test,cm

In [17]:
def log(X_train,X_test,Y_train):
    classifier = LogisticRegression(random_state =0)
    classifier.fit(X_train,Y_train)
    classifier,accuracy,report,X_test,Y_test,cm = cm_pred(classifier,X_test)
    return classifier,accuracy,report,X_test,Y_test,cm

def svm(X_train,X_test,Y_train):
    classifier = SVC(kernel='linear',random_state =0)
    classifier.fit(X_train,Y_train)
    classifier,accuracy,report,X_test,Y_test,cm = cm_pred(classifier,X_test)
    return classifier,accuracy,report,X_test,Y_test,cm

def dtree(X_train,X_test,Y_train):
    classifier = DecisionTreeClassifier(criterion ='gini',max_features ='sqrt',splitter ='best',random_state =0)
    classifier.fit(X_train,Y_train)
    classifier,accuracy,report,X_test,Y_test,cm = cm_pred(classifier,X_test)
    return classifier,accuracy,report,X_test,Y_test,cm

def random(X_train,X_test,Y_train):
    classifier =RandomForestClassifier(n_estimators =10, criterion ='entropy', random_state =0)
    classifier.fit(X_train,Y_train)
    classifier,accuracy,report,X_test,Y_test,cm = cm_pred(classifier,X_test)
    return classifier,accuracy,report,X_test,Y_test,cm

def nb(X_train,X_test,Y_train):
    classifier = GaussianNB()
    classifier.fit(X_train,Y_train)
    classifier,accuracy,report,X_test,Y_test,cm = cm_pred(classifier,X_test)
    return classifier,accuracy,report,X_test,Y_test,cm

def knn(X_train,X_test,Y_train):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train, Y_train)
    classifier,accuracy,report,X_test,Y_test,cm=cm_pred(classifier,X_test)
    return  classifier,accuracy,report,X_test,Y_test,cm

In [18]:

def rfe_classification(acclog,accsvm,accdtree,accrandom,accnb,accknn):
    rfe_df =pd.DataFrame(index =['Logistic','SVM','DecisionTree','RandomForest',],columns = ['Logistic','SVM','DecisionTree','RandomForest','NavieBayes','KNN'])
    for number,idex in enumerate(rfe_df.index):
        rfe_df['Logistic'][idex]=acclog[number]
        rfe_df['SVM'][idex]=accsvm[number]
        rfe_df['DecisionTree'][idex]=accdtree[number]
        rfe_df['RandomForest'][idex]=accrandom[number]
        rfe_df['NavieBayes'][idex]=accnb[number]
        rfe_df['KNN'][idex]=accknn[number]

    return rfe_df

In [20]:
indep_X = dataset.drop('classification_yes',axis =1)
dep_Y = dataset['classification_yes']
rfelist,selected_features= rfefeature(indep_X,dep_Y,10)
acclog=[]
accsvm=[]
accdtree=[]
accrandom=[]
accnb=[]
accknn=[]

for i in rfelist:
    X_train,X_test,Y_train,Y_test = split_scalar(i,dep_Y)
    classifier,accuracy,report,X_test,Y_test,cm =log(X_train,X_test,Y_train)
    acclog.append(accuracy)
    classifier,accuracy,report,X_test,Y_test,cm =svm(X_train,X_test,Y_train)
    accsvm.append(accuracy)
    classifier,accuracy,report,X_test,Y_test,cm =dtree(X_train,X_test,Y_train)
    accdtree.append(accuracy)
    classifier,accuracy,report,X_test,Y_test,cm =random(X_train,X_test,Y_train)
    accrandom.append(accuracy)
    classifier,accuracy,report,X_test,Y_test,cm =nb(X_train,X_test,Y_train)
    accnb.append(accuracy)
    classifier,accuracy,report,X_test,Y_test,cm =knn(X_train,X_test,Y_train)
    accknn.append(accuracy)
    
result = rfe_classification(acclog,accsvm,accdtree,accrandom,accnb,accknn)
result

C:\Users\USER 10\anaconda3\envs\datasci\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USER 10\anaconda3\envs\datasci\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

LogisticRegression()
RFE(estimator=LogisticRegression(), n_features_to_select=10)


C:\Users\USER 10\anaconda3\envs\datasci\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USER 10\anaconda3\envs\datasci\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

SVC(kernel='linear', random_state=0)
RFE(estimator=SVC(kernel='linear', random_state=0), n_features_to_select=10)
DecisionTreeClassifier(max_features='sqrt', random_state=0)
RFE(estimator=DecisionTreeClassifier(max_features='sqrt', random_state=0),
    n_features_to_select=10)
RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)
RFE(estimator=RandomForestClassifier(criterion='entropy', n_estimators=10,
                                     random_state=0),
    n_features_to_select=10)


C:\Users\USER 10\AppData\Local\Temp\ipykernel_12752\2860076751.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rfe_df['Logistic'][idex]=acclog[number]
C:\Users\USER 10\AppData\Local\Temp\ipykernel_12752\2860076751.py:5: FutureWarning: Ch

,Logistic,SVM,DecisionTree,RandomForest,NavieBayes,KNN
Logistic,0.975,0.975,0.975,0.983333,0.975,0.983333
SVM,0.983333,0.983333,0.983333,0.983333,0.975,0.983333
DecisionTree,0.983333,0.991667,0.966667,0.966667,0.975,0.983333
RandomForest,0.983333,0.983333,0.933333,1.0,0.966667,0.975


In [21]:
print("Selected Features:")
for model_name, features in selected_features.items():
    print(f"{model_name}: {features}")

Selected Features:
LogisticRegression: ['al', 'su', 'hrmo', 'sg_b', 'sg_c', 'sg_d', 'rbc_normal', 'htn_yes', 'dm_yes', 'appet_yes']
SVC: ['al', 'sc', 'hrmo', 'sg_b', 'sg_c', 'sg_d', 'rbc_normal', 'htn_yes', 'dm_yes', 'appet_yes']
DecisionTreeClassifier: ['bgr', 'hrmo', 'pcv', 'wc', 'rc', 'sg_c', 'sg_d', 'htn_yes', 'dm_yes', 'pe_yes']
RandomForestClassifier: ['al', 'bgr', 'bu', 'sc', 'hrmo', 'pcv', 'rc', 'sg_d', 'htn_yes', 'dm_yes']


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[10,50,100], 
              'criterion':['gini', 'entropy', 'log_loss'],# paramters are random forest from website
              'max_features':['sqrt', 'log2']}

grid = GridSearchCV(RandomForestClassifier(),parameters, refit = True, verbose =3, n_jobs=-1, scoring='f1') 
#refit true is for finding the best model,
#false will give last model
#https://scikit-learn.org/stable/modules/model_evaluation.html - for scoring 
grid.fit(X_train,Y_train) # can able to call directly without training and testing 

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 50, 100]},
             scoring='f1', verbose=3)

In [23]:
print(grid.best_params_)

{'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 10}


In [24]:
classifier = grid.cv_results_ 
classifier
print('The value for best parameter: ',format(grid.best_params_))

The value for best parameter:  {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 10}


In [25]:
table = pd.DataFrame.from_dict(classifier)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.066760,0.013695,0.011193,2.638092e-03,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.956522,0.957746,0.923077,0.985915,0.971429,0.958938,0.020867,18
1,0.238863,0.045479,0.022789,7.463142e-03,gini,sqrt,50,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.971429,0.972222,0.939394,0.971429,0.985507,0.967996,0.015272,13
2,0.562077,0.073192,0.023187,3.308667e-03,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.985915,0.972222,0.939394,0.971429,0.985507,0.970893,0.016932,5
3,0.060767,0.019457,0.012792,6.108966e-03,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.956522,0.985915,0.955224,0.971429,1.000000,0.973818,0.017219,1
4,0.202884,0.018352,0.014192,2.038170e-03,gini,log2,50,"{'criterion': 'gini', 'max_features': 'log2', ...",0.971429,0.972222,0.939394,0.985915,0.985507,0.970893,0.016932,5
5,0.400770,0.026854,0.020588,1.019862e-03,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.971429,0.972222,0.939394,0.985915,0.985507,0.970893,0.016932,5
6,0.046768,0.001716,0.008795,1.468413e-03,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.971429,0.957746,0.970588,0.971429,0.971429,0.968524,0.005399,11
7,0.209477,0.008892,0.014791,1.326238e-03,entropy,sqrt,50,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.971429,0.972222,0.939394,0.971429,0.985507,0.967996,0.015272,13
8,0.435849,0.032549,0.022561,5.728747e-03,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.971429,0.972222,0.939394,0.985915,0.985507,0.970893,0.016932,5
9,0.048372,0.005311,0.008396,1.019525e-03,entropy,log2,10,"{'criterion': 'entropy', 'max_features': 'log2...",0.985915,0.958904,0.939394,0.971429,1.000000,0.971128,0.021018,4


In [26]:
grid_pred = grid.predict(X_test)
grid_pred

array([0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0])

In [27]:
from sklearn.metrics import confusion_matrix
con_mat = confusion_matrix(Y_test,grid_pred)
con_mat

array([[44,  1],
       [ 1, 74]])

In [28]:
from sklearn.metrics import classification_report
class_report = classification_report(Y_test,grid_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        45
           1       0.99      0.99      0.99        75

    accuracy                           0.98       120
   macro avg       0.98      0.98      0.98       120
weighted avg       0.98      0.98      0.98       120



In [29]:
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 10}: 0.9833333333333333


In [30]:
features

['al', 'bgr', 'bu', 'sc', 'hrmo', 'pcv', 'rc', 'sg_d', 'htn_yes', 'dm_yes']

In [32]:
al_input=float(input("AL:"))
bgr_input=float(input("BGR:"))
bu_input=float(input("BU:"))
sc_input=float(input("SC:"))
hrmo_input=float(input("HRMO:"))
pcv_input=float(input("PCV:"))
rc_input=float(input("RC:"))
sg_d_input=float(input("SG:"))
htn_yes_input=float(input("HTN 0 or 1:"))
dm_yes_input=float(input("DM 0 or 1:"))

AL: 1666
BGR: 787
BU: 56
SC: 876
HRMO: 67
PCV: 899
RC: 1
SG: 0
HTN 0 or 1: 0
DM 0 or 1: 1


In [33]:
Future_Prediction=grid.predict([[al_input, bgr_input, bu_input, sc_input, hrmo_input, pcv_input, rc_input,sg_d_input,
                                 htn_yes_input, dm_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


In [34]:
import pickle
filename= "Final_model_random_forest_classifier.sav"
pickle.dump(grid,open(filename,'wb'))
load_model=pickle.load(open("Final_model_random_forest_classifier.sav",'rb'))

In [35]:
def get_categorical_prediction(prediction):
    return "Yes" if prediction == 1 else "No"
al_input=float(input("AL:"))
bgr_input=float(input("BGR:"))
bu_input=float(input("BU:"))
sc_input=float(input("SC:"))
hrmo_input=float(input("HRMO:"))
pcv_input=float(input("PCV:"))
rc_input=float(input("RC:"))
sg_d_input=float(input("SG:"))
htn_yes_input=float(input("HTN 0 or 1:"))
dm_yes_input=float(input("DM 0 or 1:"))

AL: 54
BGR: 766
BU: 465
SC: 677
HRMO: 87
PCV: 33
RC: 0
SG: 1
HTN 0 or 1: 0
DM 0 or 1: 1


In [36]:
Final_result = grid.predict([[al_input, bgr_input, bu_input, sc_input, hrmo_input, pcv_input, rc_input,sg_d_input,
                                 htn_yes_input, dm_yes_input]])
Final_result
future_prediction_categorical = get_categorical_prediction(Final_result)
future_prediction_categorical

'Yes'